# Penjelasan lengkap — Multi-Query (konseptual, kenapa, bagaimana) + tabel perbandingan

---

# 1) Inti konsep — Apa itu Multi-Query?

**Multi-Query** = membuat beberapa *query* (variasi/pecahan) dari satu pertanyaan pengguna, lalu melakukan retrieval (pencarian) secara paralel untuk tiap query. Hasil semua retrieval digabungkan, diolah, dan diserahkan ke LLM untuk disintesis menjadi jawaban akhir.

Tujuan: **meningkatkan cakupan (recall) dan kualitas konteks** yang diberikan ke LLM sehingga jawaban lebih lengkap dan akurat.

---

# 2) Kenapa (alasan/masalah yang diselesaikan)

* **Query tunggal seringkali ambigu / terlalu umum.** Satu phrasing bisa melewatkan aspek penting.
* **Satu perspektif → blind spot.** Pertanyaan bisa memiliki beberapa dimensi (definisi, proses, contoh, implikasi) — satu query mungkin hanya menangkap beberapa dimensi.
* **Melipatgandakan sinyal relevan.** Berbagai phrasing menarik dokumen yang berbeda → sinergi informasi.
* **Robustness terhadap phrasing pengguna.** Paraphrase atau ekspansi membuat retrieval tidak tergantung satu formulasi kalimat.
* **Lebih baik untuk tugas kompleks.** Untuk QA multi-step, rangkuman, analisis, dsb.

Trade-off utama: **biaya & latensi naik** (lebih banyak query → lebih banyak operasi retrieval + post-processing).

---

# 3) Varian Multi-Query (tipe query yang biasa dipakai)

1. **Paraphrase queries** — buat variasi kalimat yang sama makna.
   Contoh: `"Bagaimana sistem imun merespon virus?"` → `"Apa tahapan respon imun terhadap infeksi virus?"`

2. **Focused sub-queries (decomposition)** — pecah pertanyaan kompleks menjadi sub-soal.
   Contoh: `Main Q = "Bagaimana X mempengaruhi Y?"` → Q1: "Apa mekanisme X?" Q2: "Apa bukti empiris pengaruh X→Y?" Q3: "Apa studi kasus?"

3. **Keyword expansion / query expansion** — tambahkan istilah kunci, sinonim, akronim.
   Contoh: `virus → "viral infection", "pathogen", "antiviral"`

4. **Role or intent queries** — minta perspektif berbeda: “explain like I’m 5”, “peer-reviewer perspective”, “practitioner checklist”.

5. **Clarifying sub-queries** — model menghasilkan pertanyaan lanjutan yang menjelaskan konteks (mirip active clarification, kadang butuh pengguna).

---

# 4) Proses end-to-end (langkah detail)

Berikut alur teknisnya, langkah per langkah:

1. **Input**: Pertanyaan pengguna (Q₀).

2. **Query Understanding & Generation**

   * Tokenize / parse Q₀.
   * Pilih strategi (paraphrase, decomposition, expansion, dll).
   * Gunakan LLM atau template engine untuk menghasilkan Q₁..Qₙ.

3. **Retrieval (parallel)**

   * Untuk setiap Qi, jalankan pencarian di vector store (embedding similarity) atau hybrid (BM25 + embeddings).
   * Parameter: top_k per query (mis. k = 5–10).
   * Jalankan paralel untuk mengurangi latensi wall-clock.

4. **Aggregation & Deduplication**

   * Gabungkan semua dokumen.
   * Hapus duplikasi (by text hash atau cosine sim > threshold).
   * Tambahkan metadata source (id, doc score, query asal).

5. **Reranking / Filtering (opsional tapi disarankan)**

   * Cross-encoder untuk re-score gabungan (lebih akurat, costlier).
   * MMR (Maximal Marginal Relevance) untuk mengurangi redundansi dan memaksimalkan novelty.
   * Terapkan metadata filters (time, author, domain).

6. **Context Construction (prompt building)**

   * Pilih top N dokumen hasil rerank (batas konteks LLM).
   * Buat prompt: instruksi synthesize, sumber dikutip, batas panjang, target format (summary, steps, code, dsb).

7. **LLM Synthesis**

   * LLM menerima konteks terpilih & prompt → menghasilkan jawaban.
   * Strategi: chain-of-thought internal (jika diperlukan), atau structured outputs (bullet points + source citations).

8. **Post-processing**

   * Attach provenance (sumber + skor).
   * Confidence scoring, fallback if low confidence (minta klarifikasi atau beri opsi “I’m not sure”).
   * Cache query results / final answer.

9. **Return to User**

   * Output answer + sumber referensi (link/ID + relevansi).

Diagram singkat (text):

```
User Q → Query Generator → {Q1, Q2, Q3}
   ↘         ↘         ↘
 VectorStore VectorStore VectorStore (parallel)
   ↓           ↓           ↓
 Docs1       Docs2       Docs3 → Aggregate → Dedup → Rerank → Select topN → LLM → Answer
```

---

# 5) Teknik penting & pilihan desain (detail)

* **Vector store & embeddings**: pilih model embedding yang kuat untuk domain (sentence-transformers, OpenAI embeddings).
* **Hybrid retrieval**: gabungkan sparse (BM25) + dense (embeddings) untuk manfaat keduanya.
* **Reranker**: cross-encoder (BERT/DeBERTa) untuk akurasi re-scoring.
* **MMR**: saat butuh info yang beragam, MMR membantu memilih dokumen yang saling melengkapi.
* **Chunking**: dokumen panjang perlu dipotong (chunk) tapi simpan link ke source.
* **Paraphrase generator**: bisa pakai LLM untuk buat 3–7 paraphrase.
* **Dedup threshold**: cosine > 0.9–0.95 dianggap duplikat.
* **Parallelism & batching**: penting untuk performa — jalankan retrieval untuk setiap query secara async.
* **Cache**: simpan hasil retrieval untuk Q yang serupa/berulang.
* **Cost control**: batasi jumlah query & reranker calls; gunakan cheaper retrievers + rerank hanya top-N.

---

# 6) Contoh pseudo-kode (sederhana)

```python
# pseudo-code
queries = generate_queries(user_question, strategy="paraphrase+decompose", n=5)

# parallel retrieval
all_docs = []
for q in parallel(queries):
    docs = vectorstore.search(q, top_k=8)
    for d in docs:
        d.origin_query = q
    all_docs.extend(docs)

# dedup + merge
merged = deduplicate_and_merge(all_docs, sim_threshold=0.92)

# rerank (cross-encoder) on merged
reranked = cross_encoder_rerank(merged, user_question)
selected = select_top(reranked, n=6)

# optionally apply MMR to reduce redundancy
final_context = mmr_select(selected, user_question, k=4)

# build prompt + call LLM
prompt = build_prompt(user_question, final_context)
answer = llm.generate(prompt)

return answer, provenance=extract_provenance(final_context)
```

---

# 7) Evaluasi & metrik

Ukur dengan:

* **Precision@k** (proporsi dokumen relevan di top k),
* **Recall@k** (berapa banyak info relevan ditemukan),
* **MRR** (Mean Reciprocal Rank),
* **F1 / ROUGE** untuk jawaban vs gold answer,
* **Human eval** (fluency, accuracy, completeness),
* **Latency & cost** (ms & $ per query).

Untuk eksperimen, lakukan ablation: bandingkan single-query baseline vs multi-query variants.

---

# 8) Rekomendasi praktis singkat

* Mulai dengan **3–5 queries** (paraphrase + 1 decomposed subquery).
* Pilih **top_k = 5–8** per query, lalu dedup.
* Gunakan reranker hanya pada gabungan (mis. rerank top 50) untuk menyeimbangkan cost.
* Terapkan **MMR** jika jawabannya sering redundant.
* Cache hasil retrieval untuk query yang mirip.
* Selalu sertakan **sumber** di jawaban jika memungkinkan.

---

# 9) Tabel Perbandingan — Single Query vs Multi-Query Variants

| Strategi                                    | Coverage (Recall) |           Precision (Noise) |                   Redundansi |       Latency |         Cost | Kompleksitas Implementasi | Cocok untuk                                |
| ------------------------------------------- | ----------------: | --------------------------: | ---------------------------: | ------------: | -----------: | ------------------------: | ------------------------------------------ |
| **Single Query (baseline)**                 |     Rendah–sedang |               Sedang–tinggi |                       Rendah |        Rendah |       Rendah |                    Rendah | FAQ, simple factual QA                     |
| **Multi-Query (paraphrase)**                |     Sedang–tinggi |                      Sedang |                       Sedang |        Sedang |       Sedang |                    Sedang | Ketika phrasing pengguna bervariasi        |
| **Multi-Query (expansion / synonyms)**      |            Tinggi | Rendah–sedang (lebih noise) |                       Tinggi |        Sedang |       Sedang |                    Sedang | Topik luas dengan banyak terminologi       |
| **Multi-Query (decomposition/sub-queries)** |     Sangat tinggi |     Tinggi (setelah rerank) | Rendah (bila digabung smart) |  Lebih tinggi | Lebih tinggi |                    Tinggi | Pertanyaan kompleks / multi-step reasoning |
| **Multi-Query (role/intents)**              |     Sedang–tinggi |                      Sedang |                       Sedang | Sedang–tinggi |       Sedang |                    Sedang | Butuh perspektif (practitioner vs layman)  |
| **Multi-Query + Rerank + MMR**              |            Tinggi |                      Tinggi |                       Rendah |        Tinggi |       Tinggi |                    Tinggi | Sistem production QA / penelitian ilmiah   |

**Catatan**:

* *Coverage* = kemungkinan menemukan semua informasi relevan.
* *Precision* = seberapa banyak hasil relevan (noise = non-relevan).
* *Redundansi* = seberapa banyak hasil overlap / ulangan.
* *Latency & Cost* naik sejalan jumlah query dan reranker usage.

---

# 10) Kesimpulan ringkas

* **Gunakan multi-query saat kebutuhan informasi kompleks atau phrase variance tinggi.**
* **Kontrol biaya dan kualitas** dengan reranker, MMR, dedup, dan batching.
* **Eksperimen & ukur**: lakukan A/B antara single vs multi query di domain kamu untuk menemukan trade-off optimal.

---



In [7]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
urls = [
    "https://lilianweng.github.io/posts/2021-03-21-lm-toxicity/",
    "https://lilianweng.github.io/posts/2024-02-05-human-data-quality/",
    "https://lilianweng.github.io/posts/2020-06-07-exploration-drl/",
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
]

loader=WebBaseLoader(
    web_path=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs=loader.load()

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

hf_api_key = os.getenv("HF_API_KEY")
groq_api_key=os.getenv('GROQ_API_KEY')

Pertanyaan yang **bagus banget 🔥**, karena untuk *text cleaning via LLM pipeline*, model yang kamu pilih akan sangat berpengaruh pada:

* **Kualitas hasil cleaning** (apakah dia benar-benar membersihkan noise, typo, dan HTML dengan baik)
* **Biaya & kecepatan inferensi**
* **Kemampuan multibahasa / domain tertentu**

---

## 🧠 Tujuan: *Text Cleaning via LLM Pipeline*

Tujuan utama pipeline ini:

> Membersihkan teks dari noise (HTML, whitespace, kalimat rusak, duplikasi, dll) sambil tetap mempertahankan makna aslinya.

---

## 🚀 Best Practice — Model Pilihan Berdasarkan Kebutuhan

| Tujuan / Fokus                                           | Model yang Direkomendasikan                                 | Kelebihan                                                         | Catatan                                           |
| -------------------------------------------------------- | ----------------------------------------------------------- | ----------------------------------------------------------------- | ------------------------------------------------- |
| **Cepat dan ringan (CPU-friendly)**                      | `google/flan-t5-base` / `flan-t5-large`                     | Bagus untuk normalisasi teks ringan                               | Sudah kamu pakai — cocok untuk preprocessing umum |
| **Multilingual / Bahasa campuran (Inggris + Indonesia)** | `facebook/mbart-large-50-many-to-many-mmt`                  | Bisa memahami banyak bahasa, termasuk Indonesia                   | Butuh GPU untuk performa optimal                  |
| **Kualitas tinggi, gaya natural dan robust**             | `mistralai/Mixtral-8x7B-Instruct-v0.1` (via API atau lokal) | Cleaning-nya lebih kontekstual (bisa perbaiki grammar, coherence) | Terlalu besar untuk lokal CPU                     |
| **Berfokus pada kualitas teks ilmiah / artikel panjang** | `google/t5-large-ssm-nq`                                    | Training pada natural questions → hasil cleaning lebih koheren    | Bagus untuk teks akademik                         |
| **Berbasis open-weight dan hemat resource**              | `csebuetnlp/mT5_m2m_cleaning`                               | Dibuat khusus untuk text normalization multibahasa                | Model kecil dan cepat (mirip flan-t5)             |

---

## 🧩 Best Practice Pipeline (Reusable Function)

Berikut contoh *best practice* function `text_cleaner()` yang siap dipakai:

```python
from transformers import pipeline
from tqdm import tqdm
import pandas as pd
from langchain.schema import Document

# Pilih model terbaik sesuai kebutuhan
llm_cleaner = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",  # bisa diganti sesuai rekomendasi tabel
    device_map="auto"
)

def text_cleaner(docs, model=llm_cleaner, max_input_len=4000):
    """
    Membersihkan teks dari daftar Document (LangChain) menggunakan LLM.
    Output: DataFrame dengan kolom sumber dan hasil cleaning.
    """
    results = []

    for doc in tqdm(docs, desc="🧹 Cleaning text with LLM"):
        raw_text = doc.page_content.strip()
        prompt = (
            "Clean and normalize the following text by removing HTML tags, noise, "
            "and redundant whitespace, but keep the original meaning:\n\n"
            + raw_text[:max_input_len]
        )
        try:
            output = model(prompt, max_length=1024, truncation=True)[0]["generated_text"]
        except Exception as e:
            output = f"[ERROR cleaning text: {e}]"

        results.append({
            "source": doc.metadata.get("source", "unknown"),
            "original_length": len(raw_text),
            "cleaned_length": len(output),
            "cleaned_text": output
        })

    return pd.DataFrame(results)
```

---

## 💡 Tips Pemakaian

* Kalau kamu mau **multilingual cleaning**, gunakan `facebook/mbart-large-50-many-to-many-mmt` atau `mT5`.
* Kalau teks sangat panjang (ribuan kata), bisa pakai **chunking sebelum cleaning**.
* Untuk efisiensi, kamu bisa *cache* hasil cleaning ke `.parquet` supaya tidak perlu dijalankan ulang.
* Hindari model yang terlalu “creative” (seperti GPT-style) untuk cleaning — karena bisa mengubah makna asli teks.

---


In [13]:
import re
import html
from bs4 import BeautifulSoup
from tqdm import tqdm
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline

llm_cleaner = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",  
)

def rule_based_clean(text: str) -> str:
    text = html.unescape(text)
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[\r\n\t]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"[^\w\s.,!?;:'\"()%\-]", "", text)
    return text.strip()

def text_cleaner(docs, model=None, use_llm=False):
    cleaned_docs = []
    for doc in tqdm(docs, desc="🧹 Cleaning Docs (Fast Mode)"):
        raw = doc.page_content.strip()
        cleaned = rule_based_clean(raw)
        if use_llm and model is not None:
            prompt = (
                "Refine and improve readability of this cleaned text "
                "without changing its meaning:\n\n" + cleaned
            )
            try:
                cleaned = model(prompt, max_length=512, truncation=True)[0]["generated_text"]
            except Exception as e:
                cleaned = f"[ERROR refining text: {e}]"
        cleaned_docs.append(Document(page_content=cleaned, metadata=doc.metadata))
    return cleaned_docs

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)



Device set to use cpu


In [14]:
docs = text_splitter.split_documents(docs)
cleaned_docs = text_cleaner(docs, model=llm_cleaner, use_llm=False)

🧹 Cleaning Docs (Fast Mode): 100%|██████████| 222/222 [00:00<00:00, 6228.58it/s]
